### Reference 
- [Rasa Masterclass - episode 7](https://www.youtube.com/watch?v=EuOyncigHLA)
- [Rasa Docs - Policies](https://rasa.com/docs/rasa/policies/)
- Vlasov, V., Mosig, J. E., & Nichol, A. (2019). Dialogue transformers. <i>arXiv preprint arXiv:1910.00486</i>.

## Dialogue Policy 
Policy: 사용자의 input에 어시스턴트가 취할 행동을 predict하는 것과 관련<br>

**config.yml** 파일에서 설정
- 보통 policy name과 a set of parameters로 구성됨

<img src="img\rasa20.png" width="400" height="400">


### 0. Hyper parameters
많은 policy에서 보편적으로 사용하는 parameters 

1. max_history
    - How many conversational turns an assistant looks at when learning to make the predictions for the next action
    - max_history: n
        - n: 어시스턴트가 기억하고 있어야 하는 training stories and patterns의 길이를 나타냄
        - n을 너무 큰 값으로 설정하면, 모델이 커져 학습하는데 많은 시간이 소요됨
        - 특정한 dialogue turn을 고려하고 싶은 경우, max_history 대신 slot으로 설정하면 됨 
    - 예: intent가 out-of-scope인 경우의 training story
    
    <img src="img\rasa21.png" width="400" height="400">
<br>     
2. data_augmentation
    - 짧은 story들을 합쳐 더 긴 형태의 story를 만드는 것 
    - 모델의 performance 향상, 불필요한 history of conversation 무시, 특정 상황에서 자주 쓰이는 response 예측할 수 있음
    - augmentation flag를 통해 조정할 수 있음
    - 예: rasa train -- augmentation 20 → 200 augmented stories created 
    - augmented story를 사용하면 전체 training story 수가 증가하여 training time이 증가함. 따라서 이미 training data가 충분히 많고 성능이 좋다면 augmentation을 사용할 필요가 없음
    
    <img src="img\rasa28.png" width="400" height="400">

### 1. Memorization Policy 
- 가장 간단하고 기본적인 policy 
- 학습한 story들을 바탕으로 어시스턴트의 다음 행동을 predict 
- 단독으로 사용하지 않고 보통 다른 policy와 함께 사용
- 2가지 parameter가 있음 
    - max_history
    - priority
        - the priority of the policy를 설정
        - default로 놔두는 것을 추천 

### 2. Mapping Policy 
<img src="img\rasa24.png" width="300" height="300">
<br>

- 특정 intent에 대해 항상 특정 행동이 따라오는 경우 사용하기 좋은 policy 
- 다른 policy와 함께 사용해야 함
    - 특정 intent와 맵핑된 특정 행동이 실행된 후 다음 행동을 예측하기 위해선 다른 policy가 필요하기 때문
    - 함께 사용되는 다른 policy는 mapping policy에 의해 예측된 행동을 고려하여 그 다음 행동을 예측함
    - UserUtteranceReverted를 통해 맵핑된 행동을 제거할 수있음. 다른 policy들이 다음 행동을 예측할 때 맵핑된 행동을 고려하지 않게 됨
- 설정 방법
    1. domain.yml: 특정 intent가 예측되었을 때 어떤 행동이 triggered 되어야 하는지 설정
    2. config.yml: policy 추가

<img src="img\rasa22.png" width="400" height="400">

### 3. Keras Policy 
- Keras라는 딥러닝 라이브러리의 neural network를 사용하여 어시스턴트의 다음 행동을 predict
- last action, intent or entities of current user input, slot, previous states of dialogue 등을 고려하여 행동을 결정할 수 있음
- training story로부터 학습함. training data가 충분하다면 challenging situation에 잘 대응할 수 있으며 자연스러운 대화를 진행해나갈 수 있음 
- 다른 policy들과 함께 사용
- 디폴트 모델은 LSTM이며, 필요에 따라 customize 할 수 있음 
- 4가지 parameter가 있음 
    - max_history
    - epochs: the number of times the algorithm will see the training data (one epoch = one forward pass and one backward pass of all training examples)
    - validation_split: the amount of data used evaluating the model during the training 
    - random_seed: 같은 input으로부터 reproducible results를 얻을 수 있도록 도와줌. 이를 위해서 integer number로 설정 
    
<img src="img\rasa23.png" width="500" height="500">

### 4. Embedding Policy (TEDP) 
- Multi-turn dialogue modeling에 있어 다른 policy보다 성능이 뛰어남  
- 패턴을 학습하고 적절한 다음 행동을 predict함에 있어 RNN 대신 Transformer를 사용함 
    - Transformer의 장점
        - simpler architecture
        - faster model
        - better with achieving lower perplexity across different corpora
        - better with unexpected user inputs like chitchat
- policy parameter은 [링크](https://rasa.com/docs/rasa/policies#ted-policy)에서 확인할 수 있음

    
<img src="img\rasa25.png" width="500" height="500">


<center>(현재 dialogue 상태와 모든 가능한 system action을 비교하여 가장 유사성이 높은 행동을 선택하는 것)</center>

### 5. Form Policy 
- 어시스턴트가 특정 행동을 하기 전에 특정 정보를 알아내야 하는 경우가 있음 
    - 예를 들어 medicare locator assistant의 경우, 사용자의 주소, 이름, 나이 등을 알아야 함
- 특정 slot에 저장해야 하는 정보를 유도하는 form action을 predict하고자 할 때 사용
- 필요한 모든 slot이 채워질때까지 계속해서 form action을 진행

### 6. Fallback Policy 
- 어시스턴트가 사전에 학습하지 않은 input을 받게 될 경우 이에 대처하는 방법
- NLU 또는 Dialogue Management Model의 threshold를 설정한 후, 각 모델의 예측이 이를 넘지 못하였을 때 fallback action이 예측됨 
    - fallback action의 예: Sorry I didn't get that, Can you rephrase it? 
- 4가지 parameter가 존재
    - nlu_threshold: NLU의 예측을 받아들이기 위한 최소한의 confidence 
    - core_threshold: Dialogue Management Model의 예측을 받아들이기 위한 최소한의 confidence 
    - ambiguity_threshold: minimum amount by which the confidence of the top intent prediction must excced the confidence of second highest predction
    - fallback_action_name: fallback이 예측되었을 때 실행될 행동 
       - rasa의 디폴트 action을 사용하거나, 자신만의 action을 만들 수도 있음(이 경우 이 action을 domain과 stories 파일에도 추가시켜야 함)

### 7. TwoStageFallback Policy 
- Two-stage intent affirmation process을 진행
    1. First prediction
        - If the NLU prediction confidence thresholds are not met, an assitant will ask to user a specific intent
            - If the user affirms the intent, the conversation continues
            - If the user denies the intent, an assitant asks to rephrase the message

    2. Rephrasing
        - If the classification on rephrased message is confident, the conversation continues
        - Otherwise, an assitant asks to rephrase the intent. and it will provide a suggestion of what an assitant thinks the user actually meant. 

    3. Second affirmation
        - If the user affirms the intent, the conversation continues
        - If the users denies the intent, the fallback action is triggered


- 6가지 parameter가 존재 
    1. nlu_threshold
    2. ambiguity_threshold
    3. core_threshold
    4. fallback_core_action_name: confidence of Rasa Core action이 threshold보다 낮을 경우 어시스턴트가 하게 되는 행동
    5. fallback_nlu_action_name: confidence of NLU intent prediction이 threshold보다 낮을 경우 어시스턴트가 하는 되는 행동
    6. deny_suggestion_intent_name: 사용자의 intent를 알아내는 것에 실패하였을 때 사용자의 intent로 설정하게 되는 것
    
<img src="img\rasa27.png" width="500" height="500">

##### **Fallback Policy와 TwoStageFallback Policy 중 하나를 선택**

### 8. Policy Priority
- 보통 2개 이상의 policy를 같이 사용함. 이때 어시스턴트의 다음 행동을 결정함에 있어 confidence score가 가장 높은 policy의 행동을 선택함
- 만약 policy들의 confidence score가 같을 경우, policy priority를 통해 어떤 policy의 행동을 선택할 것인지 결정
    - rasa에서 제공하는 default policy priority는 어시스턴트가 가장 feasible하게 행동할 수 있도록 설정되어 있음 
    - default policy priority를 사용하고 반드시 필요할 경우에만 수정하는 것이 좋음
    - higher number - higher priority

<img src="img\rasa26.png" width="500" height="500">